In [ ]:
# !pip install sdv

In [1]:
from sdv.tabular import GaussianCopula
import pandas as pd


In [2]:
data = pd.read_csv('audience.csv', nrows=705)

data = data[data.columns[56:]]
data.head()


,Audience credibility,Country 1,Country 1 %,Country 2,Country 2 %,Country 3,Country 3 %,Country 4,Country 4 %,Country 5,...,Audience report interest 21,Audience report interest 21 %,Audience report interest 22,Audience report interest 22 %,Audience report interest 23,Audience report interest 23 %,Audience report interest 24,Audience report interest 24 %,Audience report interest 25,Audience report interest 25 %
0,0.754,United States,0.382855,Brazil,0.069794,Nigeria,0.047205,United Kingdom,0.033594,South Africa,...,Shopping & Retail,0.116552,Luxury Goods,0.097586,Jewellery & Watches,0.077586,Gaming,0.073793,Business & Careers,0.063103
1,0.8403,United States,0.169507,Germany,0.151931,Italy,0.114780,France,0.053129,United Kingdom,...,Beauty & Cosmetics,0.133186,"Home Decor, Furniture & Garden",0.126706,Business & Careers,0.099545,Jewellery & Watches,0.079967,Activewear,0.049635
2,0.7831,Portugal,0.719775,Brazil,0.135087,United States,0.018031,Spain,0.014780,United Kingdom,...,"Home Decor, Furniture & Garden",0.080943,Shopping & Retail,0.073258,Gaming,0.064037,Luxury Goods,0.052254,Jewellery & Watches,0.049693
3,0.884788,Portugal,0.857821,Brazil,0.017416,United Kingdom,0.016466,France,0.016149,Spain,...,Activewear,0.079090,Shopping & Retail,0.076619,Luxury Goods,0.061295,Gaming,0.059812,Jewellery & Watches,0.059318
4,0.7875,United States,0.259875,Brazil,0.134254,Mexico,0.063026,Italy,0.053097,United Kingdom,...,Shopping & Retail,0.097368,Activewear,0.080756,Luxury Goods,0.068490,Jewellery & Watches,0.066190,"Home Decor, Furniture & Garden",0.062356


In [3]:
model = GaussianCopula()

model.fit(data)

/home/apres/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)
/home/apres/anaconda3/lib/python3.7/site-packages/scipy/stats/_continuous_distns.py:639: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/home/apres/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/home/apres/anaconda3/lib/python3.7/site-packages/scipy/stats/_continuous_distns.py:5320: RuntimeWarning: divide by zero encountered in true_divide
  return c**2 / (c**2

In [4]:
model.save('model.pkl')


In [13]:
# new_data_df_list=[]
# for i in range(100):
#     new_data_df_list.append(model.sample(1000))

In [16]:
# pd.concat(new_data_df_list).to_csv('1M.csv', index=False)

In [4]:
# Socialtalk
fulldata = pd.read_csv('audience.csv')
no_required_rows = fulldata.shape[0] - data.shape[0]

In [5]:
new_data = model.sample(no_required_rows)

In [9]:
new_data.to_csv('audience_imputed_data.csv', index=False)

# Evaluate data quality

In [30]:
from sdv.evaluation import evaluate

evaluate(new_data,data)

In [ ]:
0.5859857015609296

## Performing Chi test and KS test

In [9]:
evaluate(new_data,data, metrics=['CSTest', 'KSTest'],aggregate=False)

,metric,name,raw_score,normalized_score,min_value,max_value,goal,error
0,CSTest,Chi-Squared,0.999981,0.999981,0.0,1.0,MAXIMIZE,None
1,KSTest,Inverted Kolmogorov-Smirnov D statistic,0.907130,0.907130,0.0,1.0,MAXIMIZE,None


# Performing ML efficacy metrics test

In [26]:
new_data = new_data.drop(new_data[new_data['Audience credibility'] == 'Toys, Children & Baby'].index)
new_data.shape

(19971, 159)

In [27]:
data = data.drop(data[data['Audience credibility'] == 'Toys, Children & Baby'].index[0])
data.shape

(704, 159)

In [28]:
from sdv.metrics.tabular import MLPRegressor
MLPRegressor.compute(data, new_data, target='Audience credibility')

/home/apres/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


-535.9595961716633

In [29]:
from sdv.metrics.tabular import LinearRegression
LinearRegression.compute(data, new_data, target='Audience credibility')

/home/apres/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


-615944293312804.4